In [ ]:
"""
Author: Reggie Karssiens (2667014)
Msc Earth and Climate Sciences, Vrije Universiteit Amsterdam
Supervisor: Liam Heffernan 
Date: 29-01-2025

Aims and expected outcomes
This script provides the tool to automatically reduce the sample size of the geological cores to 10%, 25%, 50%, 75%, and 90%. The outcome is 50 shapefiles containing
random samples sizes. With these multiple reduces sample sizes the geostatistics can be taken under the look to decide what sample size is the most suitable for this methodology.
The source data includes 190 geological cores from the VU and DINOloket. These 5 sensitiviy files are used to import them in the excisting script to calculate the prediction of the C_mass and
see how the results of the interpolation with different amount of cores differ according to geostatistics. 

Data:
Available input data: (Fileformat, raster/vector, variable, unit, dimensions, resolution, projection, other relevant info adding to understand the data and import the right modules)
- 190 Geological cores containing peat depth, geometry, corenumber and source (VU or DINOloket). (shapefile)

Outcomes:
10 folders with 5x shapefiles files containing every coverage degree. Afterwards these 50 files are imported into ArcGIS PRO to use
the tool EBK with cross-validation and compare how the statistics is impacted by different sample sizes.
"""

In [ ]:
"""
## Methodology: Sample Size Sensitivity Analysis

### Objective
Determine the most suitable sample size for geostatistical interpolation of soil organic carbon mass (C_mass) in peatlands using Empirical Bayesian Kriging (EBK).

### Data and Procedure

The original dataset comprises 190 geological cores from two sources:
- 20 cores from Vrije Universiteit (VU)
- 170 cores from DINOloket national database

Five random subsamples were extracted from the original 190 cores with varying percentages:
- 10% (n = 19 cores)
- 25% (n = 48 cores)
- 50% (n = 95 cores)
- 75% (n = 143 cores)
- 90% (n = 171 cores)

All coverage degrees were generated using a dynamic random state ensure reproducibility 
across runs and to make sure different measurements are chosen. This approach guarantees that:
- Results are reproducible and verifiable
- Differences in interpolation quality reflect sample size effects, not random variation
- The spatial pattern changes enough each time the code is run to make sure it is really random
- The methodology can be replicated by other researchers

Bootstrapping is added to make sure that the mean, ranges and standard deviations are scientifically reliable.
There is a folder made for each run in the bootstrap, where in each bootstrap file five shapefile containing each coverage degree are stored.

### Rationale
By comparing interpolation results across different sample sizes, we could identify the optimal balance between data density and prediction accuracy,
ensuring the geostatistical method produces reliable C_mass distribution maps without requiring excessive field data.
"""

In [2]:
import arcpy
from arcpy.sa import *
import geopandas as gpd
from random import sample
import os

arcpy.CheckOutExtension("GeoStats")
arcpy.CheckOutExtension("Spatial")

# To allow overwriting outputs change overwriteOutput option to True. You do this to make sure that exciting file will be replaced, when running the code again.
arcpy.env.overwriteOutput = True


In [3]:
#Read the original file. 
Fieldcores190cores = r"C:\\Coding projects Msc Earth & Climate\\Research_project\\data\\spaarnwoude\\Python_result\\gdf_SOC_PD_C\\gdf_SOC_PD_C.shp"

#Make a geodataframe of it as i want change the sample size, while keeping the geometry
Fieldcores190_copy = gpd.read_file(Fieldcores190cores)

In [4]:
#Totaal aantal cores
total_cores = len(Fieldcores190_copy)
print( 'In totaal zijn er {} cores in de gdf original'.format(total_cores))

In totaal zijn er 190 cores in de gdf original


In [6]:
#This is a list with the coverage degrees groups. 
coverage_degree_in_percentage = [10, 25, 50, 75, 90]

#Inset bootstrapping. This is the amount of times that the output is made to make an average of the outcome. 
n_bootstraps = 10

#Define the starting number, with the same random seed you'll always get the random sequence, so this is increased in the for loop per coverage degree group
random_seed = 42

#Definier een output path as it loops through the list of percentages, 
#9 folders will be created in de output_dir with the content of 5 shapefile, for each degree one shapefile
output_dir = r"C:\Coding projects Msc Earth & Climate\Research_project\data\spaarnwoude\Python_intermediate\sensitivity_analysis\shapefiles_reduced_sample_sizes"
        
#In this loop the 10 files are made storing five shapefiles with associated coverage degrees (10%, 25%, 50%, 75%, 90%)
for run in range(n_bootstraps):
        
    #Make folder for by iterating through the amount of bootstraps (10x)
    output_folder = f"{output_dir}\\Fieldcores_randomsamp_bootstrap_{run+1}"
    os.makedirs(output_folder, exist_ok=True)

    #Check wether the folder is created
    print(f"Created folder: {output_folder}")

    #In this loop the 5 shapefiles of every coverage degree is created.
    for pct in coverage_degree_in_percentage:

        #Bereken het aantal cores bij dit percentage
        n_cores = int(total_cores * pct/100) 

        #Basis seed is 42, + the different coverage degrees and the time it is runned, this random state is added to make sure different spatial patterns
        #are made and not the same spatial pattern come in the results.
        gdf_sample = Fieldcores190_copy.sample(n=n_cores, random_state=42 + pct*100 + run)
        
        #This is where it's stored. Inside the map the shp is stored per coverage degree.
        output_shp = f"{output_folder}\\Fieldcores_randsamp{pct}pct.shp"

        gdf_sample.to_file(output_shp)
        
        print(f"Saved: {output_shp}")
        print(f"Verified: {len(gdf_sample)}cores saved")

        #make 10 outputs with a new random sample dataset 

        #Than 10 files and evey file should be run in ArcGIS with the EBK tool (5 files for each coverage)
        #Make 5 excel files including 10 different random sample datasets and combine them with making a chart.


print("\n" + "="*60)
print("All sample files created successfully!")
print("="*60)



Created folder: C:\Coding projects Msc Earth & Climate\Research_project\data\spaarnwoude\Python_intermediate\sensitivity_analysis\shapefiles_reduced_sample_sizes\Fieldcores_randomsamp_bootstrap_1
Saved: C:\Coding projects Msc Earth & Climate\Research_project\data\spaarnwoude\Python_intermediate\sensitivity_analysis\shapefiles_reduced_sample_sizes\Fieldcores_randomsamp_bootstrap_1\Fieldcores_randsamp10pct.shp
Verified: 19cores saved
Saved: C:\Coding projects Msc Earth & Climate\Research_project\data\spaarnwoude\Python_intermediate\sensitivity_analysis\shapefiles_reduced_sample_sizes\Fieldcores_randomsamp_bootstrap_1\Fieldcores_randsamp25pct.shp
Verified: 47cores saved
Saved: C:\Coding projects Msc Earth & Climate\Research_project\data\spaarnwoude\Python_intermediate\sensitivity_analysis\shapefiles_reduced_sample_sizes\Fieldcores_randomsamp_bootstrap_1\Fieldcores_randsamp50pct.shp
Verified: 95cores saved
Saved: C:\Coding projects Msc Earth & Climate\Research_project\data\spaarnwoude\Pyth

In [12]:
#Im going to set a workspace, so that the raster files are correctly stored

ebk_output_dir= r"C:\\Coding projects Msc Earth & Climate\\Research_project\\data\\spaarnwoude\\Python_result\\EBKofrandomsampling"

arcpy.env.workspace = ebk_output_dir
arcpy.env.scratchWorkspace = ebk_output_dir

In [17]:
#Lead the computer to the right file in the working directory to grab the shapefiles for the loop
input_dir = r"C:\Coding projects Msc Earth & Climate\Research_project\data\spaarnwoude\Python_intermediate\sensitivity_analysis\shapefiles_reduced_sample_sizes"

#Load the new layers to this folder
output_dir = ebk_output_dir

AOI_shapefile = r"C:\Coding projects Msc Earth & Climate\Research_project\data\spaarnwoude\brondata\AOI_Spaarnwoude.shp"

#With this code the computer iterates through the loop for all 5 shapefiles in the input_dir and does Bayesian kriging and stores it in the output dir
for pct in percentage:
    try:
        print(f"Processing {pct}%...")
        input_shp = os.path.join(input_dir, f"Fieldcores_randsamp{pct}pct.shp")
        output_raster = os.path.join(output_dir, f"C_mass_EBK_{pct}pct.tif")

        arcpy.ga.EmpiricalBayesianKriging(
            in_features=input_shp,
            z_field="C_mass",
            out_raster=output_raster,
            cell_size="87.24"
        )
        arcpy.management.Clip(
            in_raster=output_raster,
            output_raster=output_raster,
            in_template_dataset = AOI_shapefile,
            nodata_value=-9999,
            clipping_geometry="ClippingGeometry"
        )

        print(f'Succesfully clipped: {output_raster}')

    except Exception as e:
        print(f"✗ Error at {pct}%: {str(e)}")

print("\n" + "="*60)
print("EBK processing completed!")
print("="*60) 
    


        

Processing 10%...
✗ Error at 10%: Clip() got an unexpected keyword argument 'output_raster'
Processing 25%...
✗ Error at 25%: Clip() got an unexpected keyword argument 'output_raster'
Processing 50%...
✗ Error at 50%: Clip() got an unexpected keyword argument 'output_raster'
Processing 75%...
✗ Error at 75%: Clip() got an unexpected keyword argument 'output_raster'
Processing 90%...
✗ Error at 90%: Clip() got an unexpected keyword argument 'output_raster'

EBK processing completed!
